In [ ]:
import numpy as np
import pandas as pd
import os

dtypes_raw = {
    'elapsed_time': np.int32,
    'event_name': 'category',
    'name': 'category',
    'level': np.uint8,
    'room_coor_x': np.float32,
    "index": np.int32,
    'room_coor_y': np.float32,
    'screen_coor_x': np.float32,
    'screen_coor_y': np.float32,
    'hover_duration': np.float32,
    'text': 'category',
    'fqid': 'category',
    'room_fqid': 'category',
    'text_fqid': 'category',
    'fullscreen': 'category',
    'hq': 'category',
    'music': 'category',
    'level_group': 'category'}

In [ ]:
wd = os.getcwd()
print(wd[-10:])

# 1. Load your model: [your model name here] (in this case: steves optuna xgboost decision tree)

In [ ]:
import xgboost as xgb #delete?
import pickle

# Variant 1.: On Kaggle: Load the model from kaggle input files
#with open('/kaggle/input/best-models-dict-optunahp/best_models_dict_optunahp.pkl', 'rb') as f:
#    models = pickle.load(f)

# Variant 2.: Locally load model files
# set correct working directory
wd = os.getcwd()
print("Current working directory: ", wd)
if wd[-10:] == 'submission': wd = wd[:-11]
os.chdir(wd)
print("New working directory: ", os.getcwd())

#with open('models/best_models_dict_optunahp.pkl', 'rb') as f:
  #  models = pickle.load(f)

# 2. Run the model
DON'T READ THIS: This jo_wilder-kind-of-timeseries-API simulates the real world scenario for the model you've trained. The iter_test gives you pairs of: 
1. X_test_raw: gamelog data of a single session_id from only one level_group 
2. y_test_label_preds: and secondly a dataframe (similiar to raw/train_labels.csv) in which you have to fill in your predictions for each question of that level_group.

**Additional WARNINGS**:
- submission.csv is NOT automatically reseted, even after a Kernel reboot the iter_test will continue to add sample submissions to the submission.csv
- the iter_tester needs a kernel reboot after each run - or you will get the error: "Exception: You can only iterate over `iter_test()` once."

In [ ]:
# Unfortunately this doesn't work because you don't have the rights to do this on Kaggle:
# os.remove("/kaggle/working/submission.csv")

# 1. On Kaggle: Import utils for feature engineering
#import sys
#sys.path.insert(1, '/kaggle/input/utils-for-preprocessing')
#import data_preprocessing_steve, loader_steve, preprocessing_func
# from /kaggle/input/utils-for-preprocessing import data_preprocessing_steve, loader_steve, preprocessing_func

# 2. Locally:
from utils.data_preprocessing_steve import pp_pipeline_noah

# 1. On Kaggle:
#import jo_wilder
# 2.Locally:
import data.raw.jo_wilder
env = jo_wilder.make_env()
iter_test = env.iter_test()

limits = {'0-4':(1,4), '5-12':(4,14), '13-22':(14,19)}

for (X_test_raw, y_test_label_preds) in iter_test: # Only one session_id at a time?
    print(X_test_raw)
    # Pre-process the raw test data
    X_test = data_preprocessing_steve.pp_pipeline_noah(data=X_test_raw, file_path=None, flatten=True, saveIntermediateFiles=False,
                     dtypes=dtypes_raw, output=True)
    
    # Apparently each test data in each iteration of iter_test has only data of one level_group. We find it here and then only go through the questions of this particular level_group:
    grp = X_test.level_group.values[0]
    a,b = limits[grp]
    
    # We loop for each question - t
    for t in range(a,b):
        # Get the model for question t
        model = models[f'{t}']
        y_preds = model.predict(X_test)
        print(y_preds)
        # Make predictions
        mask = y_test_label_preds.session_id.str.contains(f'q{t}')
        r = rng.uniform()
        if r <= prob_0:
            y_test_label_preds.loc[mask,'correct'] = 0
        else:
            y_test_label_preds.loc[mask,'correct'] = 1
    
    # Add your predictions of this session_id to the submission before getting the next session_id data
    env.predict(y_test_label_preds)

In [ ]:
from utils.data_preprocessing_steve import pp_pipeline_noah
#from utils.loader_steve import load_data

X_test_raw = pd.read_csv('data/submission/X_test_raw_1.csv')
y_test_label_preds = pd.read_csv('data/submission/y_test_label_preds_1.csv')

# Pre-process the raw test data
X_test_dict = pp_pipeline_noah(data=X_test_raw, file_path=None, flatten=True, saveIntermediateFiles=False,
                 dtypes=dtypes_raw, output=True)

# Apparently each test data in each iteration of iter_test has only data of one level_group. We find it here and then only go through the questions of this particular level_group:
grp = list(X_test_dict.keys())[0]
X_test = X_test_dict[grp]

limits = {'0-4':(1,4), '5-12':(4,14), '13-22':(14,19)}
a,b = limits[grp]

# We loop for each question - t
for t in range(a,b):
    # Get the model for question t
    model = models[f'{t}']
    y_preds = model.predict(X_test)
    print(y_preds)
    # Make predictions
    mask = y_test_label_preds.session_id.str.contains(f'q{t}')
    r = rng.uniform()
    if r <= prob_0:
        y_test_label_preds.loc[mask,'correct'] = 0
    else:
        y_test_label_preds.loc[mask,'correct'] = 1

# Add your predictions of this session_id to the submission before getting the next session_id data

# 3. Quick submission EDA


In [ ]:
df = pd.read_csv('submission.csv')
print(df.shape)
print(df.head(20))
print(df.correct.mean())

In [ ]:
## the end result is a submission file containing all test session predictions
! head submission.csv

In [ ]:
## the end result is a submission file containing all test session predictions
! head submission.csv